In [2]:
#Good Practive:
!rm -rf /opt/conda/lib/python3.10/site-packages/fsspec*
!pip install fsspec==2024.9.0 --force-reinstall --no-deps

  Using cached fsspec-2024.9.0-py3-none-any.whl.metadata (11 kB)
Using cached fsspec-2024.9.0-py3-none-any.whl (179 kB)


In [3]:
#Kaggle Files
rd = './kaggle-files'

In [4]:
#test
